In [38]:
from dataPreparation import user_item
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedKFold,train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, mean_squared_error
from scipy.sparse import csr_matrix
from math import sqrt

In [3]:
# split data
train_set, test_set = train_test_split(user_item, test_size=0.30, random_state=100)

In [4]:
train_set_aux=csr_matrix(train_set.values)
train_set_aux

<20475x29 sparse matrix of type '<class 'numpy.int64'>'
	with 24818 stored elements in Compressed Sparse Row format>

In [5]:
#model 1

In [6]:
similarity=cosine_similarity(train_set_aux)

In [7]:
def prediction(users, similarity):
    pred = similarity.dot(users) / np.array([np.abs(similarity).sum(axis=1)]).T
    return pred

In [8]:
recommendations=prediction(train_set.values, similarity)

In [9]:
print(recommendations)

[[7.99221835e-03 1.62009510e+00 7.84470218e-05 ... 5.54704211e-05
  6.77972587e-04 2.70629464e-03]
 [1.62169157e-02 1.30692813e-01 0.00000000e+00 ... 1.14855449e-04
  7.67441870e-04 1.04037982e-03]
 [1.62169157e-02 1.30692813e-01 0.00000000e+00 ... 1.14855449e-04
  7.67441870e-04 1.04037982e-03]
 ...
 [1.62169157e-02 1.30692813e-01 0.00000000e+00 ... 1.14855449e-04
  7.67441870e-04 1.04037982e-03]
 [1.62169157e-02 1.30692813e-01 0.00000000e+00 ... 1.14855449e-04
  7.67441870e-04 1.04037982e-03]
 [1.03063814e-02 1.73319582e-01 0.00000000e+00 ... 4.99365371e-04
  1.53397592e-03 2.38030727e-03]]


In [10]:
pd.DataFrame(recommendations).head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.007992,1.620095,0.000078,0.000078,0.000068,0.000617,0.041915,0.001540,0.005453,0.008860,...,0.006397,0.002264,0.000146,0.015726,0.000842,0.085728,0.002481,0.000055,0.000678,0.002706
1,0.016217,0.130693,0.000000,0.000094,0.000469,0.000420,0.027899,0.001710,0.004313,0.004312,...,0.007520,0.002028,0.000108,0.020987,0.000580,0.022935,0.004981,0.000115,0.000767,0.001040
2,0.016217,0.130693,0.000000,0.000094,0.000469,0.000420,0.027899,0.001710,0.004313,0.004312,...,0.007520,0.002028,0.000108,0.020987,0.000580,0.022935,0.004981,0.000115,0.000767,0.001040
3,0.016217,0.130693,0.000000,0.000094,0.000469,0.000420,0.027899,0.001710,0.004313,0.004312,...,0.007520,0.002028,0.000108,0.020987,0.000580,0.022935,0.004981,0.000115,0.000767,0.001040
4,0.010026,0.443214,0.000000,0.000000,0.000351,0.000000,0.044159,0.002781,0.007584,0.007527,...,0.006288,0.002514,0.000574,0.014378,0.002069,1.218314,0.001244,0.000000,0.000287,0.002385


In [11]:
#Eval 1
def calculate_target_set(val_set):
    target = np.zeros(val_set.shape)
    val_set_aux = val_set.copy()
    for user in range(val_set.shape[0]):
        if(len(val_set[user, :].nonzero()[0]) > 1):
            rats = np.random.choice(val_set[user, :].nonzero()[0], replace=False)
            val_set_aux[user, rats] = 0.
            target[user, rats] = val_set[user, rats]
    return val_set_aux, target

In [32]:
test_set_b, target_set = calculate_target_set(test_set.values)

In [33]:
test_set_aux=csr_matrix(test_set_b)
test_set_aux

<8776x29 sparse matrix of type '<class 'numpy.int64'>'
	with 9175 stored elements in Compressed Sparse Row format>

In [34]:
similarity_val=cosine_similarity(test_set_aux)

In [35]:
recommendations_val=prediction(test_set_b, similarity_val)

In [36]:
sqrt(mean_squared_error(target_set[target_set.nonzero()], recommendations_val[target_set.nonzero()]))

2.1791650657193475

In [37]:
mean_squared_error(target_set[target_set.nonzero()], recommendations_val[target_set.nonzero()])

4.748760383651609

In [18]:
#model 2

In [19]:
# split data
train_set, test_set = train_test_split(user_item, test_size=0.30, random_state=2000)

In [20]:
train_set_aux=csr_matrix(train_set.values)
train_set_aux

<20475x29 sparse matrix of type '<class 'numpy.int64'>'
	with 24754 stored elements in Compressed Sparse Row format>

In [21]:
similarity_2=cosine_similarity(train_set_aux)

In [22]:
def prediction_general(user, similarity):
    rating_means = user.mean(axis=1)
    mean_diff = (user - rating_means[:,np.newaxis])
    pred = rating_means[:,np.newaxis] + similarity.dot(mean_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    return pred

In [23]:
recommendations_2 = prediction_general(train_set.values, similarity_2)

In [24]:
pd.DataFrame(recommendations_2)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,-0.025625,1.575742,-0.032688,-0.032688,-0.032535,-0.032245,0.007107,-0.031539,-0.027635,-0.023961,...,-0.026661,-0.031546,-0.032620,-0.016673,-0.032250,0.050600,-0.030289,-0.032767,-0.032305,-0.030163
1,-0.018702,0.094631,-0.035514,-0.035420,-0.035012,-0.035239,-0.006241,-0.034281,-0.032054,-0.032382,...,-0.026396,-0.033569,-0.035406,-0.012499,-0.035139,-0.013370,-0.029993,-0.035399,-0.034817,-0.034391
2,-0.025625,1.575742,-0.032688,-0.032688,-0.032535,-0.032245,0.007107,-0.031539,-0.027635,-0.023961,...,-0.026661,-0.031546,-0.032620,-0.016673,-0.032250,0.050600,-0.030289,-0.032767,-0.032305,-0.030163
3,-0.025625,1.575742,-0.032688,-0.032688,-0.032535,-0.032245,0.007107,-0.031539,-0.027635,-0.023961,...,-0.026661,-0.031546,-0.032620,-0.016673,-0.032250,0.050600,-0.030289,-0.032767,-0.032305,-0.030163
4,-0.023757,0.144867,-0.031008,-0.030438,-0.030954,-0.031008,1.446635,-0.029079,-0.026563,-0.029423,...,-0.027738,-0.029664,-0.031008,-0.019859,-0.031008,0.004812,-0.028922,-0.031008,-0.031008,-0.026220
5,-0.020378,0.401074,-0.030907,-0.030907,-0.030907,-0.030907,0.011174,-0.029506,-0.023335,-0.024014,...,-0.025704,-0.029619,-0.030331,-0.015852,-0.029138,1.184918,-0.029658,-0.030907,-0.030619,-0.027858
6,-0.018702,0.094631,-0.035514,-0.035420,-0.035012,-0.035239,-0.006241,-0.034281,-0.032054,-0.032382,...,-0.026396,-0.033569,-0.035406,-0.012499,-0.035139,-0.013370,-0.029993,-0.035399,-0.034817,-0.034391
7,0.012013,0.935217,0.000507,0.000550,0.000819,0.000873,0.035507,0.001695,0.004834,0.006684,...,0.007943,0.002015,0.000593,0.019712,0.000916,0.055929,0.004329,0.000516,0.001033,0.002393
8,-0.025625,1.575742,-0.032688,-0.032688,-0.032535,-0.032245,0.007107,-0.031539,-0.027635,-0.023961,...,-0.026661,-0.031546,-0.032620,-0.016673,-0.032250,0.050600,-0.030289,-0.032767,-0.032305,-0.030163
9,0.084746,0.198080,0.067934,0.068028,0.068437,0.068209,0.097208,0.069168,0.071394,0.071066,...,0.077053,0.069879,0.068043,0.090949,0.068310,0.090078,0.073455,0.068049,0.068632,0.069058


In [25]:
# Eval 2

In [91]:
test_set_c, target_set = calculate_target_set(test_set.values)

In [92]:
test_set_aux=csr_matrix(test_set_c)
test_set_aux

<8776x29 sparse matrix of type '<class 'numpy.int64'>'
	with 9175 stored elements in Compressed Sparse Row format>

In [93]:
similarity_eval_2=cosine_similarity(test_set_aux)

In [94]:
recommendations_eval_2=prediction_general(test_set_b, similarity_eval_2)

In [95]:
sqrt(mean_squared_error(target_set[target_set.nonzero()], recommendations_eval_2[target_set.nonzero()]))

1.9969915764157362

In [96]:
mean_squared_error(target_set[target_set.nonzero()], recommendations_eval_2[target_set.nonzero()])

3.987975356275407